In [137]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [138]:
import pandas as pd

# Carregar os DataFrames a partir dos arquivos CSV
df = pd.read_csv('test.csv')
dft = pd.read_csv('todosGrupos.csv')

# Ver as colunas de cada DataFrame
print("Colunas do DataFrame 'df':")
print(df.columns)

print("\nColunas do DataFrame 'dft':")
print(dft.columns)


Colunas do DataFrame 'df':
Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID'],
      dtype='object')

Colunas do DataFrame 'dft':
Index(['mileage', 'mileage_group', 'id_mileage_group', 'id_transmissao',
       'transmission_type', 'body_type', 'id_body_type', 'modelo',
       'id_modelos', 'marca', 'id_categoria_marca', 'ano', 'categoria_decadas',
       'id_categoria_decadas', 'city', 'id_city', 'insurance', 'id_insurance',
       'seats', 'tipo_carro', 'id_tipo_carro', 'engine_capacity',
       'engine_capacity_group', 'id_engine_capacity_group', 'fuel_type',
       'id_fuel_type', 'kms_driven', 'kms_group', 'id_kms_group', 'max_power',
       'max_power_group', 'id_max_power_group', 'owner_type', 'id_owner_type',
       'resale_price_Lakh'],
      dtype='object')


In [139]:
# KMS __________________________________________________
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()
df = df.astype({
   
    'kms_driven': float,
   
})

In [140]:
# groups kms

limites = [0, 50000, 150000, 300000, 600000, 900000, float('inf')]  
rotulos = ['Até 50.000 km', '50.000 km a 150.000 km', '150.000 km a 300.000 km', '300.000 km a 600.000 km', '600.000 km a 900.000 km', 'Acima de 900.000 km']

df['kms_group'] = pd.cut(df['kms_driven'], bins=limites, labels=rotulos, right=False)

df.loc[df['kms_driven'] == 1200000, 'kms_group'] = 'Acima de 900.000 km'

In [141]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['kms_group'], dft['id_kms_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_kms_group'] = df['kms_group'].map(mapeamento_max_power_group)

In [142]:
# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition


In [143]:
# Calcular a moda da coluna 'seats'
moda_seats = df['seats'].mode()[0]

# Substituir os valores nulos na coluna 'seats' pela moda
df['seats'].fillna(moda_seats, inplace=True)



In [144]:
# Substituir os valores 'Third Party insurance' por 'Third Party' em df
df['insurance'] = df['insurance'].replace('Third Party insurance', 'Third Party', regex=True)

# Substituir os valores NaN por 'No Insurance' em df
df['insurance'].fillna('No Insurance', inplace=True)

# Visualizar a coluna 'insurance' atualizada em df
print(df['insurance'].unique())
print(dft['id_insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
[0.2557372  1.30339183 0.44746999 1.41626911 3.33882157]


In [145]:

print(df['insurance'].unique())
print(dft['insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
['Third Party' 'Zero Dep' 'Comprehensive' 'Not Available' 'No Insurance']


In [146]:
# Mapear os valores desejados em 'df' para os valores correspondentes em 'dft'
mapeamento_valores = {
    'Third Party insurance': 'Third Party',
    pd.NA: 'No Insurance'
}

# Aplicar as transformações nos valores da coluna 'insurance' em 'df'
df['insurance'] = df['insurance'].replace(mapeamento_valores).fillna('No Insurance')

# Visualizar os valores únicos após as transformações em 'df'
print(df['insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']


In [147]:
# ENGINE __________________________________________________
#remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

df = df.astype({
    
    'engine_capacity': float,
  
})

# groups engine_capacity

limites = [0,1, 1000, 2000, 3000, 4000, float('inf')]  
rotulos = ['Elétrico', 'até 1000 cc', 'até 2000 cc', 'até 3000 cc', 'até 4000 cc', 'mais de 4000 cc']

df['engine_capacity_group'] = pd.cut(df['engine_capacity'], bins=limites, labels=rotulos, right=False)



In [148]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['engine_capacity_group'], dft['id_engine_capacity_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_engine_capacity_group'] = df['engine_capacity_group'].map(mapeamento_max_power_group)

In [149]:
# MAX POWER 
# #convert to bhp__________________________________________________
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

#Create grupos

# groups max_power


limites = [0, 100, 200, 300, 400, 500, 600]  
rotulos = ['0-100 bhp', '100-200 bhp', '200-300 bhp', '300-400 bhp', '400-500 bhp', '500-600 bhp']

df['max_power_group'] = pd.cut(df['max_power'], bins=limites, labels=rotulos, right=False)

In [150]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['max_power_group'], dft['id_max_power_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_max_power_group'] = df['max_power_group'].map(mapeamento_max_power_group)


In [151]:
df['max_power_group'] = df['max_power_group'].astype('string')
df['engine_capacity_group'] = df['engine_capacity_group'].astype('string')
print(df.dtypes)


full_name                           object
registered_year                     object
engine_capacity                    float64
insurance                           object
transmission_type                   object
kms_driven                         float64
owner_type                          object
fuel_type                           object
max_power                          float64
seats                              float64
mileage                             object
body_type                           object
city                                object
ID                                   int64
kms_group                         category
id_kms_group                       float64
engine_capacity_group       string[python]
id_engine_capacity_group           float64
max_power_group             string[python]
id_max_power_group                 float64
dtype: object


In [152]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [153]:
#MILEAGE

novo_valor = dft['mileage'].mean()

#assign value to blank lines
df['mileage'] = df['mileage'].fillna(novo_valor)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto

In [154]:
df = df.astype({
   
    'mileage': float,
})

In [155]:
#groups seats

import pandas as pd



# Definindo os limites e rótulos para os grupos de automóveis
limites_lugares = [1, 3.9, 5.9, 7.9, float('inf')]  # Usando float('inf') para incluir todos os valores superiores a 7.9
rotulos_lugares = ['Carro Desportivo', 'Carro Compacto', 'Carro Familiar', 'Carrinha/Transporte']

# Criando uma nova coluna 'tipo_carro' com base nos grupos definidos
df['tipo_carro'] = pd.cut(df['seats'], bins=limites_lugares, labels=rotulos_lugares, right=False)

# Exibindo as primeiras linhas para verificar o resultado
print(df[['seats', 'tipo_carro']].head())


# groups mileaage
# Define os limites para os grupos de forma crescente
limites_mileage = [0, 10.000, 30.000, 50.000, 70.000, 90.000, float('inf')]  # Usando float('inf') para incluir todos os valores restantes

# Define os rótulos para os grupos
rotulos_mileage = ['0-10k km', '10k-30k km', '30k-50k km', '50k-70k km', '70k-90k km', '90k+ km']

# Cria a nova coluna 'mileage_group' com base nos limites e rótulos definidos
df['mileage_group'] = pd.cut(df['mileage'], bins=limites_mileage, labels=rotulos_mileage, right=False)

   seats      tipo_carro
0    5.0  Carro Compacto
1    5.0  Carro Compacto
2    5.0  Carro Compacto
3    5.0  Carro Compacto
4    5.0  Carro Compacto


In [156]:
#ANO
# Extract the year and create a new column
df['ano'] = df['full_name'].str.extract(r'(\d{4})')

# Display the resulting DataFrame
print(df['ano'])

0       2017
1       2020
2       2010
3       2010
4       2018
        ... 
3485    2022
3486    2017
3487    2017
3488    2016
3489    2017
Name: ano, Length: 3490, dtype: object


In [157]:
import pandas as pd

# Convertendo a coluna 'ano' para o tipo numérico (se estiver representada como texto)
df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

# Determinando o início das décadas e o valor máximo para categorização de 10 em 10 anos
inicio_decadas = 1910
fim_decadas = df['ano'].max()
limites_decadas = list(range(inicio_decadas, int(fim_decadas) + 11, 10))

# Adicionando um limite extra se o último ano ultrapassar o último limite
if limites_decadas[-1] < fim_decadas:
    limites_decadas.append(int(fim_decadas))

# Ajustando os rótulos para corresponder aos limites
rotulos_decadas = [f'{lim}-{lim+9}' for lim in limites_decadas[:-1]]

# Criando uma nova coluna 'categoria_decadas' com base nos grupos de décadas
df['categoria_decadas'] = pd.cut(df['ano'], bins=limites_decadas, labels=rotulos_decadas, right=False)

# Exibindo as primeiras linhas para ver o resultado
print(df[['ano', 'categoria_decadas']].head())


    ano categoria_decadas
0  2017         2010-2019
1  2020         2020-2029
2  2010         2010-2019
3  2010         2010-2019
4  2018         2010-2019


In [158]:
dft.columns

Index(['mileage', 'mileage_group', 'id_mileage_group', 'id_transmissao',
       'transmission_type', 'body_type', 'id_body_type', 'modelo',
       'id_modelos', 'marca', 'id_categoria_marca', 'ano', 'categoria_decadas',
       'id_categoria_decadas', 'city', 'id_city', 'insurance', 'id_insurance',
       'seats', 'tipo_carro', 'id_tipo_carro', 'engine_capacity',
       'engine_capacity_group', 'id_engine_capacity_group', 'fuel_type',
       'id_fuel_type', 'kms_driven', 'kms_group', 'id_kms_group', 'max_power',
       'max_power_group', 'id_max_power_group', 'owner_type', 'id_owner_type',
       'resale_price_Lakh'],
      dtype='object')

In [159]:
# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['mileage'], dft['id_mileage_group']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_mileage_group'] = df['mileage'].map(mapeamento_marca)

In [160]:
# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['tipo_carro'], dft['id_tipo_carro']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_tipo_carro'] = df['tipo_carro'].map(mapeamento_marca)

In [161]:
# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['categoria_decadas'], dft['id_categoria_decadas']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_categoria_decadas'] = df['categoria_decadas'].map(mapeamento_marca)

In [162]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,...,id_engine_capacity_group,max_power_group,id_max_power_group,tipo_carro,mileage_group,ano,categoria_decadas,id_mileage_group,id_tipo_carro,id_categoria_decadas
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.0,Comprehensive,Manual,53658.0,First Owner,Petrol,81.8,5.0,...,0.125127,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2017,2010-2019,0.008712,4.116973,0.112823
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.0,Not Available,Automatic,33055.0,Second Owner,Petrol,81.8,5.0,...,0.125127,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2020,2020-2029,0.008712,4.116973,0.724804
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.0,Third Party,Manual,90385.0,Third Owner,Petrol,67.1,5.0,...,0.767307,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2010,2010-2019,0.008712,4.116973,0.112823
3,2010 Maruti Alto LXi,2010,796.0,Third Party,Manual,110000.0,First Owner,Petrol,46.3,5.0,...,0.767307,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2010,2010-2019,0.008712,4.116973,0.112823
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.0,Comprehensive,Manual,58000.0,Second Owner,Petrol,81.8,5.0,...,0.125127,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2018,2010-2019,0.008712,4.116973,0.112823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,NaN,Third Party,Automatic,30000.0,First Owner,Electric,73.7,5.0,...,4.116973,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2022,2020-2029,NaN,4.116973,0.724804
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.0,Third Party,Manual,64024.0,First Owner,Petrol,67.0,5.0,...,0.767307,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2017,2010-2019,0.008712,4.116973,0.112823
3487,2017 Honda City i VTEC VX,Dec 2017,1497.0,Comprehensive,Manual,39000.0,First Owner,Petrol,117.3,5.0,...,0.125127,100-200 bhp,0.457437,Carro Compacto,10k-30k km,2017,2010-2019,0.008712,4.116973,0.112823
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.0,Comprehensive,Manual,33000.0,First Owner,Petrol,88.7,5.0,...,0.125127,0-100 bhp,0.199520,Carro Compacto,10k-30k km,2016,2010-2019,0.008712,4.116973,0.112823


In [163]:
import pandas as pd

def substituir_valores(df, coluna_alvo, mapeamento):
    df[coluna_alvo] = df[coluna_alvo].replace(mapeamento).fillna(df[coluna_alvo])



# Criar os mapeamentos desejados para cada coluna
mapeamento_id_cidade = dict(zip(dft['city'], dft['id_city']))
mapeamento_id_body_type = dict(zip(dft['body_type'], dft['id_body_type']))
mapeamento_id_owner_type = dict(zip(dft['owner_type'], dft['id_owner_type']))
mapeamento_id_fuel_type = dict(zip(dft['fuel_type'], dft['id_fuel_type']))
mapeamento_id_insurance = dict(zip(dft['insurance'], dft['id_insurance']))



# Chamar a função para substituir os valores nas respectivas colunas
substituir_valores(df, 'city', mapeamento_id_cidade)
substituir_valores(df, 'body_type', mapeamento_id_body_type)
substituir_valores(df, 'owner_type', mapeamento_id_owner_type)
substituir_valores(df, 'fuel_type', mapeamento_id_fuel_type)
substituir_valores(df, 'insurance', mapeamento_id_insurance)


# Visualizar o DataFrame 'df' atualizado
print(df)




                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [164]:
import pandas as pd
import re

# Supondo que já carregou os DataFrames df e dft

# Função para extrair a marca a partir da coluna 'full_name' em df
def extrair_marca(nome):
    match = re.search(r'\d{4}\s(.+?)\s', nome)  # Expressão regular para pegar a marca após o ano
    if match:
        return match.group(1)
    else:
        return None

# Aplicar a função para extrair a marca e criar uma nova coluna 'marca' em df
df['marca'] = df['full_name'].apply(extrair_marca)

# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['marca'], dft['id_categoria_marca']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_categoria_marca'] = df['marca'].map(mapeamento_marca)

# Visualizar o DataFrame df atualizado
print(df)



                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [165]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_categoria_marca'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_categoria_marca'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [166]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_owner_type_dft = dft['id_owner_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_owner_type_dft + 1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['owner_type'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [167]:
#soluçoes id_ENGINE

import pandas as pd

def adicionar_0_1_maior_valorENG(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_engine_dft = dft['id_mileage_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_engine_dft + 1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_mileage_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valorENG(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [168]:
#soluçoes id_ENGINE

import pandas as pd

def adicionar_0_1_maior_valorENG(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_engine_dft = dft['id_engine_capacity_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_engine_dft + 1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_engine_capacity_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valorENG(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [169]:
#soluçoes id_maxpower

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_max_power_dft = dft['id_max_power_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_max_power_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [170]:
#soluçoes marca

import pandas as pd
import re

# Supondo que já carregou o DataFrame df

# Função para extrair o ano dos primeiros quatro caracteres de 'full_name' em df
def extrair_ano(nome):
    match = re.match(r'\d{4}', nome)  # Expressão regular para pegar os primeiros quatro dígitos (ano)
    if match:
        return match.group()
    else:
        return None

# Aplicar a função para extrair o ano e criar uma nova coluna 'ano' em df
df['ano'] = df['full_name'].apply(extrair_ano)

# Visualizar o DataFrame df com a nova coluna 'ano'
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [171]:
# Contar os valores nulos (NaN) em cada coluna
contagem_nan = df.isna().sum()

# Exibir a contagem de valores nulos em cada coluna
print(contagem_nan)

full_name                    0
registered_year             15
engine_capacity              5
insurance                    0
transmission_type            0
kms_driven                   0
owner_type                   0
fuel_type                    0
max_power                   13
seats                        0
mileage                      0
body_type                    0
city                         0
ID                           0
kms_group                    0
id_kms_group                 0
engine_capacity_group        5
id_engine_capacity_group     0
max_power_group             13
id_max_power_group           0
tipo_carro                   0
mileage_group                0
ano                          0
categoria_decadas            0
id_mileage_group             0
id_tipo_carro                0
id_categoria_decadas         0
marca                        0
id_categoria_marca           0
dtype: int64


In [172]:

#soluçoes id_body_type

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_body_type' em dft
    maior_valor_id_max_power_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Valores a substituir e seus respectivos substitutos
    substitute_values = {
        'Tata': novo_valor,
        'Mahindra': novo_valor,
        'Maruti': novo_valor,
        'Audi': novo_valor,
        'Toyota': novo_valor,
        'Honda': novo_valor,
        'BMW': novo_valor,
        'Datsun': novo_valor,
        'Chevrolet': novo_valor,
        'Jaguar': novo_valor,
        'Porsche': novo_valor
    }

    # Substituir os valores na coluna 'body_type' pelos valores do dicionário substitute_values
    df['body_type'].replace(substitute_values, inplace=True)



# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance transmission_type  kms_driven  owner_type  \
0              1197.0   0.447470            Manual     53

In [173]:
# Obtendo os valores únicos da coluna 'body_type'
valores_unicos_body_type = df['body_type'].unique()

# Exibindo os valores únicos
print(valores_unicos_body_type)


[0.35617351 0.62267905 3.73985157 0.5685834  1.35055998 2.27812373
 3.33882157 3.11697282 3.41800282]


In [174]:
df.mileage

0       18.900000
1       21.210000
2       18.200000
3       19.700000
4       17.190000
          ...    
3485    19.638759
3486    23.010000
3487    17.800000
3488    18.700000
3489    22.380000
Name: mileage, Length: 3490, dtype: float64

In [175]:
# Contar a quantidade de NaN em cada coluna do DataFrame X
nan_count = df.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)

full_name                    0
registered_year             15
engine_capacity              5
insurance                    0
transmission_type            0
kms_driven                   0
owner_type                   0
fuel_type                    0
max_power                   13
seats                        0
mileage                      0
body_type                    0
city                         0
ID                           0
kms_group                    0
id_kms_group                 0
engine_capacity_group        5
id_engine_capacity_group     0
max_power_group             13
id_max_power_group           0
tipo_carro                   0
mileage_group                0
ano                          0
categoria_decadas            0
id_mileage_group             0
id_tipo_carro                0
id_categoria_decadas         0
marca                        0
id_categoria_marca           0
dtype: int64


In [176]:
df = df.astype({
    
    'ano': float,
    'body_type': float,
    
  
})

In [177]:
#Export
#Not the target
colunas_selecionadas = ['body_type','id_categoria_marca','id_categoria_decadas','city','insurance','id_tipo_carro','id_mileage_group','id_engine_capacity_group','fuel_type','id_kms_group','id_max_power_group','owner_type','ID']

if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('testTudoGrupos.csv', index=False)

In [178]:
df_colunas_selecionadas = pd.DataFrame(colunas_selecionadas)

# Contar a quantidade de NaN em cada coluna do DataFrame df_colunas_selecionadas
nan_count = df_colunas_selecionadas.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)


0    0
dtype: int64
